In [1]:

import numpy as np
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from num2words import num2words

In [2]:
stop_words = set(stopwords.words())
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()

In [3]:
seq1 = """


Many systems provide hardware support for critical section code. The critical section problem could be solved easily in a single-processor environment if we could disallow interrupts to occur while a shared variable or resource is being modified.

In this manner, we could be sure that the current sequence of instructions would be allowed to execute in order without pre-emption. Unfortunately, this solution is not feasible in a multiprocessor environment.

Disabling interrupt on a multiprocessor environment can be time consuming as the message is passed to all the processors.

This message transmission lag, delays entry of threads into critical section and the system efficiency decreases.


"""


seq2 = """

In this page, we will learn about the basics of OOPs. Object-Oriented Programming is a paradigm that provides many concepts, such as inheritance, data binding, polymorphism, etc.

Simula is considered the first object-oriented programming language. The programming paradigm where everything is represented as an object is known as a truly object-oriented programming language.

Smalltalk is considered the first truly object-oriented programming language.



"""




seq3 = """

Object-oriented programming: As the name suggests, Object-Oriented Programming or OOPs refers to languages that uses 
objects in programming. Object-oriented programming aims to implement real-world entities like inheritance, hiding, 
polymorphism etc in programming. The main aim of OOP is to bind together the data and the functions that operate on
them so that no other part of the code can access this data except that function.


"""

seq4 = """

The OSI Model we just looked at is just a reference/logical model. It was designed to describe the functions of the communication system by dividing the communication procedure into smaller and simpler components. But when we talk about the TCP/IP model, it was designed and developed by Department of Defense (DoD) in 1960s and is based on standard protocols. It stands for Transmission Control Protocol/Internet Protocol. The TCP/IP model is a concise version of the OSI model. It contains four layers, unlike seven layers in the OSI model. The layers are:

Process/Application Layer
Host-to-Host/Transport Layer
Internet Layer
Network Access/Link Layer

This layer performs the functions of top three layers of the OSI model: Application, Presentation and Session Layer. It is responsible for node-to-node communication and controls user-interface specifications. Some of the protocols present in this layer are: HTTP, HTTPS, FTP, TFTP, Telnet, SSH, SMTP, SNMP, NTP, DNS, DHCP, NFS, X Window, LPD. Have a look at Protocols in Application Layer for some information about these protocols. Protocols other than those present in the linked article are :

HTTP and HTTPS – HTTP stands for Hypertext transfer protocol. It is used by the World Wide Web to manage communications between web browsers and servers. HTTPS stands for HTTP-Secure. It is a combination of HTTP with SSL(Secure Socket Layer). It is efficient in cases where the browser need to fill out forms, sign in, authenticate and carry out bank transactions.
SSH – SSH stands for Secure Shell. It is a terminal emulations software similar to Telnet. The reason SSH is more preferred is because of its ability to maintain the encrypted connection. It sets up a secure session over a TCP/IP connection.
NTP – NTP stands for Network Time Protocol. It is used to synchronize the clocks on our computer to one standard time source. It is very useful in situations like bank transactions. Assume the following situation without the presence of NTP. Suppose you carry out a transaction, where your computer reads the time at 2:30 PM while the server records it at 2:28 PM. The server can crash very b




"""




# Semantic Similarity using sentence2vec

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [5]:
def prepreprocess_text(doc):
    filtered_sentence = []
    doc = doc.lower()
    word_tokens = tokenizer.tokenize(doc)
    for word in word_tokens:
        if(word not in stop_words):
            if(word.isdigit()):
                try:
                    word = num2words(word)
                except:
                    pass
            filtered_sentence.append( lemmatizer.lemmatize(word) )
    return (" ".join(filtered_sentence))

In [6]:
def get_emb(sent):
    sent = prepreprocess_text(sent)
    return model.encode(sent)

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity([get_emb(seq1)], [get_emb(seq2)])[0][0])
print(cosine_similarity([get_emb(seq1)], [get_emb(seq3)])[0][0])
print(cosine_similarity([get_emb(seq2)], [get_emb(seq3)])[0][0])

0.3357401
0.44786522
0.771083


# Scoring

- Idea is to have a collection of documents which would be considered ideal(which possibly would encapsulate all different types of answers possible)
- Then for a given answer sheet a weighted average of semantic similarity would be calculated acting as suggestion score foe the answer sheet   

In [ ]:
def calculate_score(ideal_doc, curr_doc):
    ideal_doc_emb, curr_doc_emb = get_emb(ideal_doc), get_emb(curr_doc)
    
    return cosine_similarity([get_emb(ideal_doc_emb)], [get_emb(curr_doc_emb )])[0][0]

In [ ]:
def estimate_similarity(IDEAL_DIR, curr_doc):
    ideal_files = os.listdir(IDEAL_DIR)
    score = 0
    for files in ideal_files:
        with open(IDEAL_DIR+files) as f:
            text = f.read()
        
        score+=calculate_score(text, curr_doc)
    
    return score/len(ideal_files)
        

In [8]:
from transformers import pipeline
nlp = pipeline("question-answering")